# 장길진 교수님 과제 (project-2)
## 2022325105 권덕규

In [2]:
import numpy as np
import os
import random
import shutil
from pydub import AudioSegment
import os

# 1. timit_wav 폴더 내의 train 데이터를 8(train) 대 2(validation)로 분할

In [3]:
def split_folder(folder_path, split_ratio):
    entries = os.listdir(folder_path)

    #폴더 리스트 섞기
    folder_list = []
    for entry in entries:
        entry_path = os.path.join(folder_path, entry)
        if os.path.isdir(entry_path):
            folder_list.append(entry_path)
            
    random.shuffle(folder_list)
    split_index = int(len(folder_list) * split_ratio)

    # 폴더 리스트 분할
    train_folders = folder_list[:split_index]
    val_folders = folder_list[split_index:]

    return train_folders, val_folders

def save_folder(folder_list, destination):
    # 폴더 복사
    for folder_path in folder_list:
        folder_name = os.path.basename(folder_path)
        destination_path = os.path.join(destination, folder_name)
        shutil.copytree(folder_path, destination_path)

In [4]:
# 폴더 경로
root_folder_path = 'timit_wav/train'

# 파일과 폴더를 8대2로 분할
train_folders, test_folders = split_folder(root_folder_path, 0.8)

# 저장 폴더 경로
train_folder = 'train_dk'
val_folder = 'val_dk'

# 저장
save_folder(train_folders, train_folder)
save_folder(test_folders, val_folder)

# 2. EPD 사용

In [11]:
def split_audio(audio_file, start_times, end_times, output_dir, prefix, wrdphn):
    # WAV 파일 로드
    audio = AudioSegment.from_file(audio_file)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 타임스탬프를 기반으로 오디오 분할
    for i, (start, end) in enumerate(zip(start_times, end_times)):
        segment = audio[start:end]
        
        output_file = os.path.join(output_dir, f"{prefix}_{wrdphn}_{i}.wav")
        segment.export(output_file, format="wav")

In [12]:
folder_path = 'train_dk'  # 대상 폴더 경로
phn_file_list = []
wrd_file_list = []
wav_file_list = []

for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.phn'):
            file_path = os.path.join(root, file_name)
            phn_file_list.append(file_path)

for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.wrd'):
            file_path = os.path.join(root, file_name)
            wrd_file_list.append(file_path)
            
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.wav'):
            file_path = os.path.join(root, file_name)
            wav_file_list.append(file_path)     

## phn 파일 읽어온 후 타임스탬프에 따라 split

In [13]:
count = 0

for file_path in phn_file_list : 
    start_number_list = []
    end_number_list = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            numbers = line.strip().split()
            for number in numbers:
                try:
                    number = int(number)
                    if str(number).isnumeric():
                        count += 1
                        if count == 1 :
                            start_number_list.append(number)
                        elif count == 2 :
                            count = 0
                            end_number_list.append(number)
                except ValueError:
                    pass
    
    output_dir = os.path.dirname(file_path) + "/phn"
    audio_file = os.path.splitext(file_path)[0] + ".wav"    
    pre_prefix = os.path.basename(file_path)
    prefix = os.path.splitext(pre_prefix)[0]    
    wrdphn = "phn"
    split_audio(audio_file, start_number_list, end_number_list, output_dir, prefix, wrdphn)

## wrd 파일 읽어온 후 타임스탬프에 따라 split

In [9]:
count = 0

for file_path in wrd_file_list : 
    start_number_list = []
    end_number_list = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            numbers = line.strip().split()
            for number in numbers:
                try:
                    number = int(number)## phn 파일 읽어온 후 타임스탬프에 따라 split
                    if str(number).isnumeric():
                        count += 1
                        if count == 1 :
                            start_number_list.append(number)
                        elif count == 2 :
                            count = 0
                            end_number_list.append(number)
                except ValueError:
                    pass
    
    output_dir = os.path.dirname(file_path) + "/wrd"
    audio_file = os.path.splitext(file_path)[0] + ".wav"    
    pre_prefix = os.path.basename(file_path)
    prefix = os.path.splitext(pre_prefix)[0]    
    wrdphn = "wrd"
    split_audio(audio_file, start_number_list, end_number_list, output_dir, prefix, wrdphn)